How to authenticate to Google APIs?



In [ ]:
var execSync = require('child_process').execSync;
try {
    require.resolve('googleapis');
    require.resolve('google-auth-library');
} catch (e) {
    execSync('npm install googleapis google-auth-library');
}


// Load client secrets from a local file.
var path = require('path');
var fs = require('fs');
var google = require('googleapis');
var googleAuth = require('google-auth-library');

// If modifying these scopes, delete your previously saved credentials
// at ~/.credentials/calendar-nodejs-quickstart.json
var SCOPES = ['https://www.googleapis.com/auth/calendar.readonly'];
var TOKEN_DIR = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE, '.credentials');
var TOKEN_PATH = path.join(TOKEN_DIR, 'calendar-nodejs-quickstart.json');
var SECRET_PATH = path.join(TOKEN_DIR, 'client_secret.json');
var credentials = JSON.parse(fs.readFileSync(SECRET_PATH));


How to exchange token for oauth client?



In [ ]:

/**
 * Store token to disk be used in later program executions.
 *
 * @param {Object} token The token to store to disk.
 */
function storeToken(token) {
  try {
    fs.mkdirSync(TOKEN_DIR);
  } catch (err) {
    if (err.code != 'EEXIST') {
      throw err;
    }
  }
  fs.writeFile(TOKEN_PATH, JSON.stringify(token));
  console.log('Token stored to ' + TOKEN_PATH);
}

/**
 * Create an OAuth2 client with the given credentials, and then execute the
 * given callback function.
 *
 * @param {Object} credentials The authorization client credentials.
 * @param {function} callback The callback to call with the authorized client.
 */
var authorize = (scopes = SCOPES) => {
    var clientSecret = credentials.installed.client_secret;
    var clientId = credentials.installed.client_id;
    var redirectUrl = credentials.installed.redirect_uris[0];
    var auth = new googleAuth();
    var oauth2Client = new auth.OAuth2(clientId, clientSecret, redirectUrl);

    try {
        // Check if we have previously stored a token.
        var token = fs.readFileSync(TOKEN_PATH);
        oauth2Client.credentials = JSON.parse(token);
        return Promise.resolve(oauth2Client);
    } catch (e) {
        if(e.code !== 'ENOENT') {
            throw e;
        } else {
            var authUrl = oauth2Client.generateAuthUrl({
                    access_type: 'offline',
                    scope: scopes
                });
            return authorizeSelenium(authUrl)
                .then(code => new Promise((resolve, reject) => {
                    oauth2Client.getToken(code, (err, token) => {
                        if(err){
                            return reject(err);
                        } else {
                            return resolve(token);
                        }
                    });
                }))
                .then(token => {
                    oauth2Client.credentials = token;
                    storeToken(token);
                    return oauth2Client;
                });
        }
    }
};
(authorize);




How to use selenium to authorize Google access?



In [ ]:
var importer = require('../Core');

var approveSelenium = (client) => {
    return client
        .waitForVisible('#scope0 + a', 3000)
        .moveToObject('#scope0')
        .moveToObject('#submit_approve_access')
        .waitForVisible('#submit_approve_access content', 3000)
        .click('#submit_approve_access content')
        .waitForVisible('#code', 3000)
        .getValue('#code');
};

var authorizeSelenium = (authUrl) => {
    var seleniumDocker = null,
        getSeleniumServer = null,
        createWebdriverClient = null,
        client = null, 
        newToken = null;
    console.log('Authorizing: ' + authUrl);
    return importer.interpretAll([
        'selenium docker',
        'set up selenium server',
        'webdriver client node',
        'decrypt password',
        'log in Google'
    ]).then(r => {
        seleniumDocker = eval(r[0].code);
        getSeleniumServer = eval(r[1].code);
        createWebdriverClient = eval(r[2].code);
        eval(r[3].code); // loginGoogle functions use the client.addCommand
        eval(r[4].code);
        return getSeleniumServer('act-selenium')
    })
    .then(() => {
        client = createWebdriverClient('localhost', 4444);
        return client
            .url(authUrl)
            .loginGoogle();
    })
    .then(() => approveSelenium(client))
    .then(code => {
        newToken = code;
        console.log(newToken);
        return newToken;
    });
};
(authorizeSelenium);

